In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from tqdm import tqdm

options = Options()
options.headless = True
options.add_argument("--window-size=1920,10000")

driver = webdriver.Chrome(options=options)

/var/folders/y8/sg2j3k7513d450p1rpxfh4sc0000gp/T/ipykernel_77224/1105842987.py:7: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


In [2]:
product_urls = [
    ("https://www.freshful.ro/c/1-brutarie-si-patiserie/101-paine", "Paine"),
    ("https://www.freshful.ro/c/2-carne-si-peste/201-carne-de-vita-si-oaie", "Carne si Peste"),
    ("https://www.freshful.ro/c/2-carne-si-peste/202-carne-de-porc", "Carne si Peste"),
    ("https://www.freshful.ro/c/2-carne-si-peste/203-carne-de-pasare", "Carne si Peste"),
    ("https://www.freshful.ro/c/2-carne-si-peste/206-peste-si-sushi", "Carne si Peste"),
    ("https://www.freshful.ro/c/3-fructe-si-legume/301-fructe-proaspete", "Fructe si Legume"),
    ("https://www.freshful.ro/c/3-fructe-si-legume/302-legume-proaspete", "Fructe si Legume"),
    ("https://www.freshful.ro/c/3-fructe-si-legume/303-verdeturi-si-plante-aromatice", "Fructe si Legume"),
    ("https://www.freshful.ro/c/3-fructe-si-legume/304-fructe-uscate-crude", "Fructe si Legume"),
    ("https://www.freshful.ro/c/3-fructe-si-legume/305-sucuri-naturale", "Fructe si Legume"),
    ("https://www.freshful.ro/c/4-lactate-branzeturi-si-oua/401-lapte-smantana-si-branza-proaspata", "Lactate, Branzeturi si Oua"),
    ("https://www.freshful.ro/c/4-lactate-branzeturi-si-oua/402-unt-si-margarina", "Lactate, Branzeturi si Oua"),
    ("https://www.freshful.ro/c/4-lactate-branzeturi-si-oua/404-iaurt-desert-si-sana", "Lactate, Branzeturi si Oua"),
    ("https://www.freshful.ro/c/4-lactate-branzeturi-si-oua/405-oua", "Lactate, Branzeturi si Oua"),
    ("https://www.freshful.ro/c/4-lactate-branzeturi-si-oua/406-branzeturi", "Lactate, Branzeturi si Oua"),
    ("https://www.freshful.ro/c/5-mezeluri-si-ready-to-cook/501-mezeluri", "Mezeluri"),
    ("https://www.freshful.ro/c/5-mezeluri-si-ready-to-cook/502-mezeluri-feliate", "Mezeluri"),
    ("https://www.freshful.ro/c/produse-vegetale/bauturi-creme-si-grasimi-vegetale", "Produse Vegetale"),
    ("https://www.freshful.ro/c/produse-vegetale/carne-si-specialitati-vegetale", "Produse Vegetale"),
    ("https://www.freshful.ro/c/7-bauturi-si-tutun/702-apa", "Apa"),
    ("https://www.freshful.ro/c/7-bauturi-si-tutun/705-vinuri", "Vinuri"),
    ("https://www.freshful.ro/c/8-ceva-sarat/801-paste-orez-si-legume-uscate", "Paste si Orez"),
    ("https://www.freshful.ro/c/8-ceva-sarat/802-conserve", "Conserve"),
    ("https://www.freshful.ro/c/8-ceva-sarat/803-condimente-si-sosuri", "Condimente si Sosuri"),
    ("https://www.freshful.ro/c/8-ceva-sarat/804-ulei-si-otet", "Ulei si Otet"),
    ("https://www.freshful.ro/c/8-ceva-sarat/faina-zahar-si-produse-gourmet", "Faina si Zahar"),
    ("https://www.freshful.ro/c/9-ceva-dulce/903-cereale-si-mic-dejun", "Dulciuri")
]

In [3]:
from pydantic import BaseModel

class ProductBase(BaseModel):
    image_url: str
    brand: str
    category: str
    name: str
    price: str

In [4]:
def get_products(elements, category):
    return [
        ProductBase(
            image_url=element.find_element(By.CLASS_NAME, "ProductCard_image__37xLX").get_attribute("src"),
            brand=element.find_element(By.CLASS_NAME, "ProductCard_brand__fw44e").text,
            category=category,
            name=element.find_element(By.CLASS_NAME, "ProductCard_nameText__9eKNm").text,
            price=element.find_element(By.CLASS_NAME, "ProductPrice_price__u_2B0").text
        )
        for element in elements
    ]

In [5]:
products = []

for product_url, category in tqdm(product_urls):
    try:
        driver.get(f"{product_url}")
        elements = driver.find_elements(By.CLASS_NAME, 'ProductCard_root__JxJK7')
        products.extend(get_products(elements, category))
    except Exception:
        pass

    for i in range(2, 10):
        try:
            driver.get(f"{product_url}/p{i}")
            elements = driver.find_elements(By.CLASS_NAME, 'ProductCard_root__JxJK7')
            if not elements:
                break
            products.extend(get_products(elements, category))
        except Exception:
            break

  0%|          | 0/27 [00:00<?, ?it/s]

100%|██████████| 27/27 [07:23<00:00, 16.41s/it]


In [6]:
len(products)

5020

In [ ]:
driver.quit()

In [7]:
import json

products_json = [p.dict() for p in products]
with open("products.json", "w") as fp:
    json.dump(products_json, fp, indent=4, ensure_ascii=False)

/var/folders/y8/sg2j3k7513d450p1rpxfh4sc0000gp/T/ipykernel_77224/1740731803.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.0.2/migration/
  products_json = [p.dict() for p in products]


In [8]:
from collections import Counter

Counter([p.category for p in products]).most_common(20)

[('Lactate, Branzeturi si Oua', 987),
 ('Fructe si Legume', 556),
 ('Vinuri', 540),
 ('Conserve', 540),
 ('Condimente si Sosuri', 540),
 ('Faina si Zahar', 405),
 ('Dulciuri', 401),
 ('Produse Vegetale', 261),
 ('Mezeluri', 180),
 ('Paste si Orez', 179),
 ('Carne si Peste', 178),
 ('Apa', 99),
 ('Ulei si Otet', 94),
 ('Paine', 60)]

In [77]:
from collections import Counter

Counter([p.category for p in products]).most_common(20)

[('Bacanie', 1758),
 ('Lactate, branzeturi si oua', 987),
 ('Bauturi', 639),
 ('Fructe si Legume', 556),
 ('Dulciuri', 401),
 ('Mezeluri', 331),
 ('Produse-Vegetale', 261),
 ('Carne si Peste', 118),
 ('Brutarie', 110)]

In [47]:
import json
import re

from engine.models.model import Product
from tqdm import tqdm

with open("products.json", "r") as fp:
    products = json.load(fp)

In [48]:
for idx, p in tqdm(enumerate(products)):
    p["id"] = str(idx)
    p["description"] = p["brand"]
    del p["brand"]
    p["price"] = p["price"].split()[0]
    p["price"] = float(p["price"].replace(',', '.'))

5020it [00:00, 527823.47it/s]


In [49]:
parsed_products = []
for p in products:
    match = re.match(r"^(.*?)(\d+\.?\d*)?([A-Za-z]+)(?: \((.*?)\))?$", p["name"])
    if not match:
        continue

    title = match.group(1).strip()
    quantity = match.group(2)
    unit = match.group(3)

    if quantity is None:
        quantity = "1"

    try:
        p["quantity"] = str(float(quantity))
        p["measure_unit"] = unit
        parsed_products.append(Product.model_validate(p))
    except Exception as err:
        print(err)

In [50]:
products[0]

{'image_url': 'https://cdn.freshful.ro/media/cache/sylius_shop_product_thumbnail/f2/a9/b8557693cde28f2a99eabd66465c.jpg',
 'category': 'Paine',
 'name': 'Pâine albă pe vatră, feliată 300g',
 'price': 2.49,
 'id': '0',
 'description': 'San Delicio',
 'quantity': '300.0',
 'measure_unit': 'g'}

In [51]:
from google.cloud import firestore

fs_client = firestore.Client("rtc-engine-392005")

In [55]:
for product in tqdm(parsed_products):
    product_ref = fs_client.collection("products").document(product.id)
    product_ref.set(product.model_dump())

100%|██████████| 5002/5002 [04:02<00:00, 20.65it/s]


In [56]:
json_products = [product.model_dump() for product in parsed_products]
with open("products.json", "w") as fp:
    json.dump(json_products, fp, indent=4, ensure_ascii=False)